In this notebook we use [SX.jl](https://github.com/JuliaReach/SX.jl) to convert SpaceEx files into a `HybridSystem` whose modes are `MathematicalSystems` types.

In [1]:
using Revise, SX, MathematicalSystems

┌ Info: Recompiling stale cache file /Users/forets/.julia/compiled/v1.1/SX/qIGUA.ji for SX [2fbac1b3-cb3a-5de3-9d8f-ec145dc5cab0]
└ @ Base loading.jl:1184


### Lorenz model

In [2]:
file = "hyst/lorenz/lorenz.xml"
readsxmodel(file, raw_dict=true)

ErrorException: read 2 components, but models with more than one component are not yet implemented; try flattening the model

A current limitation of SX is that it can only handle SpaceEx models with one network component.

The model has two components, one called `main` and other called `sys`. `main` contains the dynamics, and `sys` is just a binding between the keys and variable names `x, y, z`.

In this case we can either flatten the model (instructions in the [SX docs](https://juliareach.github.io/SX.jl/latest/examples/examples.html#Remarks-1)) or remove `sys` manually. Let's just remove the `sys` component and try again.

In [6]:
file = "hyst/lorenz/lorenz_modif.xml"
M = readsxmodel(file, raw_dict=true)

Dict{String,Any} with 10 entries:
  "automaton"        => LightAutomaton{SimpleDiGraph{Int64},SimpleEdge{Int64}}(…
  "invariants"       => Array{Expr,1}[[]]
  "transitionlabels" => Set(String[])
  "variables"        => OrderedCollections.OrderedDict(:x=>Dict("local"=>false,…
  "guards"           => Array{Expr,1}[]
  "flows"            => Array{Expr,1}[[:(x' = 10 + (y - x)), :(y' = x * (28 - z…
  "assignments"      => Array{Expr,1}[]
  "switchings"       => HybridSystems.AbstractSwitching[]
  "nlocations"       => 1
  "ntransitions"     => 0

In [4]:
f = M["flows"]

1-element Array{Array{Expr,1},1}:
 [:(x' = 10 + (y - x)), :(y' = x * (28 - z) - y), :(z' = x * y - 2.6667z)]

Possible paths:

- convert into a `BlackBoxContinuousSystem` with a `@taylorize` of a `MultivariatePolynomial`
- convert into a `PolynomialContinuousSystem`

In [ ]:
using MultivariatePolynomials, DynamicPolynomials

In [ ]:
@polyvar x y z

In [ ]:
e = f[1][1].args[2]

In [ ]:
dx = Vector{Polynomial}(undef, 3)
dx[1] = (10 + (y - x))
dx[2] = x * (28 - z) - y
dx[3] = x * y - 2.6667z;

### Van der Pol

In [ ]:
file = "hyst/vanderpol/vanderpol_althoff_modif.xml"
readsxmodel(file, raw_dict=true)

In [ ]:
f = M["flows"]

### Van der Pol hybridized

In the file `out_vanderpol_hybridized_plot_modif.xml`:

- delete the `vanderpol_althoff_net` component
- delete the note

In [ ]:
file = "hyst/vanderpol/out_vanderpol_hybridized_plot_modif.xml"
H = readsxmodel(file, raw_dict=true);

In [ ]:
H["flows"]

In [ ]:
H["variables"]

In [ ]:
H_CLCCS = readsxmodel(file, ST=ConstrainedLinearControlContinuousSystem);

In [ ]:
file = "hyst/tank/out_tank_hybridized_plot_modif.xml"
H = readsxmodel(file, ST=ConstrainedLinearControlContinuousSystem);

In [ ]:
H["variables"]